In [1]:
from matplotlib import pyplot
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import statsmodels.imputation.mice as mice
import matplotlib.pyplot as plt
%matplotlib inline

## Load Data

In [2]:
train16 = pd.read_csv("train_2016_v2.csv", parse_dates=["transactiondate"])
print(train16.shape)
train16.tail()

(90275, 3)


,parcelid,logerror,transactiondate
90270,10774160,-0.0356,2016-12-30
90271,12046695,0.0070,2016-12-30
90272,12995401,-0.2679,2016-12-30
90273,11402105,0.0602,2016-12-30
90274,12566293,0.4207,2016-12-30


In [3]:
train16["transactiondateyear"] = train16["transactiondate"].dt.year
train16["transactiondatemonth"] = train16["transactiondate"].dt.month
train16["transactiondateday"] = train16["transactiondate"].dt.day

train16.head()

,parcelid,logerror,transactiondate,transactiondateyear,transactiondatemonth,transactiondateday
0,11016594,0.0276,2016-01-01,2016,1,1
1,14366692,-0.1684,2016-01-01,2016,1,1
2,12098116,-0.0040,2016-01-01,2016,1,1
3,12643413,0.0218,2016-01-02,2016,1,2
4,14432541,-0.0050,2016-01-02,2016,1,2


In [4]:
property16 = pd.read_csv("properties_2016.csv")
print(property16.shape)
property16.tail()

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(2985217, 58)


,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
2985212,168176230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2985213,14273630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2985214,168040630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2985215,168040830,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2985216,168040430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
new_train16 = train16.set_index('parcelid').join(property16.set_index('parcelid'))
new_train16.reset_index(inplace=True, drop=True)
new_train16.head()

,logerror,transactiondate,transactiondateyear,transactiondatemonth,transactiondateday,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,0.0276,2016-08-02,2016,8,2,1.0,NaN,NaN,3.0,4.0,...,NaN,NaN,245180.0,567112.0,2015.0,321932.0,7219.18,NaN,NaN,6.037113e+13
1,-0.0182,2016-08-02,2016,8,2,1.0,NaN,NaN,3.0,3.0,...,NaN,NaN,254691.0,459844.0,2015.0,205153.0,6901.09,NaN,NaN,6.037113e+13
2,-0.1009,2016-05-03,2016,5,3,1.0,NaN,NaN,2.0,3.0,...,NaN,NaN,235114.0,384787.0,2015.0,149673.0,4876.61,NaN,NaN,6.037113e+13
3,-0.0121,2016-04-05,2016,4,5,1.0,NaN,NaN,2.0,4.0,...,NaN,NaN,262309.0,437176.0,2015.0,174867.0,5560.07,NaN,NaN,6.037113e+13
4,-0.0481,2016-07-15,2016,7,15,1.0,NaN,NaN,2.0,4.0,...,NaN,NaN,232037.0,382055.0,2015.0,150018.0,4878.25,NaN,NaN,6.037113e+13


## Regression, Imputation

In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import Imputer

/usr/local/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence


In [7]:
exc = [new_train16.columns[c] for c in range(len(new_train16.columns)) if new_train16.dtypes[c] == 'O'] + ['logerror','parcelid']
col = [c for c in new_train16.columns if c not in exc]

col.remove('transactiondate')

exc_new_train16 = new_train16[col]

In [8]:
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(exc_new_train16)
new_df16 = imp.transform(exc_new_train16)

/usr/local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [10]:
df = pd.DataFrame(new_df16, columns=col)
#df["logerror"] = new_train16["logerror"]

In [11]:
#dfX0 = pd.DataFrame(boston.data, columns=boston.feature_names)
dfX = sm.add_constant(df)
dfy = new_train16["logerror"]

model = sm.OLS(dfy, dfX)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:               logerror   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     14.24
Date:                Thu, 05 Jul 2018   Prob (F-statistic):           9.13e-81
Time:                        16:01:27   Log-Likelihood:                 36977.
No. Observations:               90275   AIC:                        -7.388e+04
Df Residuals:                   90240   BIC:                        -7.355e+04
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
transactiondatey

## Predict

In [15]:
pred = result.predict(dfX)

In [21]:
pred[:10]

0    0.022014
1    0.007637
2    0.022974
3    0.020934
4    0.017501
5    0.013365
6    0.015991
7    0.024064
8    0.018872
9    0.009140
dtype: float64

## Score

In [19]:
import tqdm
def MAE(y, ypred):
    #logerror=log(Zestimate)−log(SalePrice)
    return np.sum([abs(y[i]-ypred[i]) for i in range(len(y))]) / len(y)

In [20]:
print(MAE(new_train16["logerror"], pred))

0.06844537957483443
